In [1]:
print("hi")

hi


In [19]:
import csv
%matplotlib inline
import matplotlib.pyplot as plt
#import pandas as pd
import sys        
from scipy import stats as scistats
from random import shuffle

def get_cps(seq):
    cps = 0
    num_pairs = 0
    for i in range(0,(len(seq)-6),3):
        num_pairs += 1
        cps += cp_scores[seq[i:i+6]]
    return(cps)

def get_cpb(seq):
    cps = 0
    num_pairs = 0
    for i in range(0,(len(seq)-6),3):
        num_pairs += 1
        cps += cp_scores[seq[i:i+6]]
    #print(num_pairs)
    #print(cps)
    return((cps/num_pairs) )

# codon pair score of any length sequence
def get_cps_indep(seq):
    cps_indep = 0 
    # for indep_dependent pairs scores - frame 2
    for i in range(1,(len(seq)-3),3):
        if (seq[i:i+6] not in cp_scores_indep):
            #print("Missing " + seq[i:i+6] + " from independent scores")
            continue
        cps_indep += cp_scores_indep[seq[i:i+6]]
    # for indep_dependent pairs scores - frame 3
    for i in range(2,(len(seq)-3),3):
        if (seq[i:i+6] not in cp_scores_indep):
            #print("Missing " + seq[i:i+6] + " from independent scores")
            continue
        cps_indep += cp_scores_indep[seq[i:i+6]]
    return(cps_indep)

# codon adaptation index of any length sequence
def get_cai(seq):
    # get the CAI as the geometric mean of all the codon's individual CAI values
    cai_list = []
    for i in range(0,len(seq),3):
        cai_list.append(codon_CAI[seq[i:(i+3)]])
    return(scistats.gmean(cai_list))

# read in synonymous codons
aa_positions = dict()
aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    aa_positions[row[0]] = []
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]

# read in the human orfs and make the CAI for all the codons
codon_occs = dict()
# set up the dict
for codon in (codon_to_aa_dict.keys()):
    codon_occs[codon] = 0
genes_file = open("human_CCDS_nucleotide.current.fna")
for line in genes_file:
    line = line.rstrip()
    if line[0] != ">": 
        if (len(line)%3 != 0):
            #print("PROBLEM, genes file is not multiple of 3 per line.")
            continue
        # get the frequency of each codon
        for i in range(0,len(line),3):
            codon_occs[line[i:(i+3)]] += 1

# get the CAI of each codon
codon_CAI = dict()
for aa in (aa_to_syn_codons_dict.keys()):
    highest_frequency = 0
    for codon in (aa_to_syn_codons_dict[aa]):
        if (codon_occs[codon] > highest_frequency):
            highest_frequency = codon_occs[codon]
    for codon in (aa_to_syn_codons_dict[aa]):
        if (highest_frequency == 0):
            codon_CAI[codon] = 0
        else:
            codon_CAI[codon] = codon_occs[codon]/highest_frequency
            
cp_scores = dict()
cp_scores_file = open("human_cps_coleman.csv")
cp_reader = csv.reader(cp_scores_file, delimiter='\t')
for line in cp_reader:
    cp_scores[line[1]] = float(line[5])


cp_scores_indep = dict()
cp_scores_file = open("human_FIndep_Scores.csv")
cp_reader = csv.reader(cp_scores_file, delimiter=',')
for line in cp_reader:
    cp_scores_indep[line[0]] = float(line[1])
            
            
# read in the gene's codon sequence
codons_file = open("polio_genomics.fsa")
original_codons = ""
for line in codons_file:
    line = line.rstrip()
    if line[0] != ">":
        original_codons = original_codons + line
codons_file.close()

original_AA = ""
for i in range(0,len(original_codons),3):
    original_AA += codon_to_aa_dict[original_codons[i:i+3]]
    
recoded_sequence = "GGTGCTCACGAAAACTCGAATCGCGCGTACGGCTCGTCGACGATCAACTACACGACGATCAACTACTATCGCGATTCGGCGTCGAACGCGGCGTCGAAACAAGACTTCTCGCAAGACCCGTCGAAATTTACCGAACCGATTAAAGATGTTTTAATAAAAACGGCGCCGATGTTAAATTCGCCGAACATAGAAGCTTGCGGTTATTCGGATCGCGTATTACAACTTACGCTGGGCAACTCGACGATCACCACGCAAGAAGCGGCGAACTCGGTCGTCGCGTACGGTCGATGGCCAGAATATTTACGCGACAGTGAGGCGAATCCGGTCGATCAACCAACCGAACCGGACGTCGCGGCGTGCCGATTTTATACGCTCGATACGGTCTCCTGGACGAAAGAATCTCGAGGCTGGTGGTGGAAACTACCGGACGCGTTACGCGACATGGGCCTCTTTGGACAAAACATGTACTACCACTACTTGGGTCGATCCGGTTATACCGTACATGTTCAGTGCAACGCGTCGAAATTTCATCAAGGTGCTTTGGGTGTTTTCGCGGTACCGGAGATGTGCCTGGCGGGGGACTCGAATACGACGACGATGCACACGTCGTACCAAAACGCGAACCCAGGCGAAAAGGGGGGCACCTTCACGGGCACCTTCACGCCGGACAACAACCAGACGTCGCCGGCGCGACGATTCTGCCCGGTCGATTATTTACTTGGAAACGGTACGCTGCTGGGGAACGCGTTCGTATTTCCTCATCAAATAATAAATTTACGAACCAACAATTGCGCGACGCTTGTTTTACCGTACGTCAACAGCCTCTCGATCGACTCGATGGTGAAGCACAACAACTGGGGTATCGCGATCCTCCCGCTGGCGCCGTTAAATTTCGCGAGCGAATCGTCGCCGGAGATCCCGATTACGCTCACGATCGCGCCGATGTGTTGCGAATTTAATGGTTTACGAAATATTACGCTGCCGCGATTACAAGGACTACCGGTCATGAATACGCCGGGCTCGAACCAGTACCTCACGGCGGACAACTTCCAGAGCCCTTGCGCGTTACCGGAATTCGACGTTACGCCGCCGATCGATATTCCTGGAGAAGTAAAAAACATGATGGAGCTGGCGGAGATCGATACGATGATTCCTTTCGATCTTTCCGCGACGAAAAAAAATACGATGGAGATGTATCGCGTACGATTATCCGACAAGCCGCATACCGACGATCCGATCCTCTGCCTCTCGCTCTCGCCCGCGAGCGATCCGCGCCTCTCGCATACGATGCTTGGAGAAATATTAAATTATTATACGCACTGGGCGGGCAGCCTCAAGTTCACCTTCCTCTTCTGTGGCTCGATGATGGCGACGGGGAAATTACTTGTCTCGTACGCGCCGCCGGGGGCGGACCCGCCGAAAAAGCGAAAGGAGGCGATGCTTGGAACTCATGTCATCTGGGATATCGGTTTACAAAGCAGCTGCACCATGGTGGTGCCCTGGATCTCGAATACGACGTACCGCCAGACGATCGACGATTCCTTCACGGAGGGGGGCTACATCTCGGTCTTCTACCAAACGCGAATCGTCGTACCGCTCTCGACGCCGCGCGAAATGGACATTCTTGGTTTCGTATCGGCCTGCAATGACTTCTCGGTACGATTATTACGCGACACCACGCATATCGAACAAAAAGCGCTGGCGCAAGGACTTGGACAAATGCTGGAGTCGATGATCGACAACACGGTACGCGAAACGGTTGGTGCCGCGACGTCGCGCGACGCGTTACCGAATACGGAGGCGTCGGGGCCCACGCACTCGAAAGAAATACCGGCGCTCACGGCGGTCGAAACGGGCGCGACGAACCCGCTTGTTCCAAGCGATACCGTACAAACGCGACATGTTGTACAACATCGATCGCGAAGTGAATCGTCGATCGAAAGTTTTTTCGCGAGGGGGGCCTGTGTCACGATCATGACGGTCGACAACCCGGCGTCGACGACGAACAAAGATAAACTTTTCGCGGTCTGGAAGATCACGTACAAAGATACCGTACAATTACGACGAAAATTAGAATTCTTCACGTACTCGCGATTCGATATGGAGCTCACCTTTGTTGTTACCGCGAACTTCACGGAGACGAACAATGGACATGCGCTCAATCAAGTATATCAAATAATGTACGTACCGCCGGGGGCGCCGGTACCGGAGAAGTGGGATGACTACACGTGGCAGACGTCGTCGAACCCGTCGATCTTCTACACGTACGGTACCGCGCCGGCGCGAATATCGGTACCGTACGTGGGCATCTCGAACGCGTACTCGCACTTTTACGACGGCTTCTCGAAAGTTCCTTTAAAAGATCAATCGGCGGCGCTGGGGGACTCGTTATACGGGGCGGCGTCGCTCAACGATTTCGGTATTCTCGCGGTACGAGTAGTAAATGACCACAACCCGACGAAAGTCACGTCGAAAATTCGCGTATATTTAAAACCGAAACATATTCGCGTATGGTGCCCGCGGCCGCCGCGCGCGGTCGCGTATTACGGTCCTGGGGTCGATTATAAAGATGGTACGCTTACGCCGCTCTCGACGAAAGACCTCACGACGTAC"

back_to_original = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (recoded_sequence[i:i+3] != original_codons[i:i+3]):
        diff_total += 1
    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
print("total number of codon diferences between original and recoded sequences " + str(diff_total))
if (original_AA == back_to_original):
    print("AA sequence remains the same, good")

print("coleman codon pair bias of the original sequence: " + str(get_cpb(original_codons)))
print("coleman codon pair bias of the recoded sequence: " + str(get_cpb(recoded_sequence)))
print("frame2&3 indep codon pair score of the original sequence: " + str(get_cps_indep(original_codons)))
print("frame2&3 indep codon pair score of the recoded sequence: " + str(get_cps_indep(recoded_sequence)))

# now, for each amino acid, find all of the positions to scramble
for i in range(0,len(recoded_sequence),3):
    aa = codon_to_aa_dict[recoded_sequence[i:i+3]]
    aa_positions[aa].append(i)

#print(aa_positions)
number_permutations = 1

best_so_far_sequence = recoded_sequence
best_so_far_indep_cps = get_cps_indep(recoded_sequence)
for p in range(0, number_permutations):
    aa_permuted_positions = dict()
    # permute all the postions
    for aa in (aa_to_syn_codons_dict.keys()):
        if (aa == "C"):
            print(aa)
        temp_list = list(aa_positions[aa])
        if (aa == "C"):
            print(temp_list)
        shuffle(temp_list)
        if (aa == "C"):
            print(temp_list)
        aa_permuted_positions[aa] = temp_list
    # construct the new string with all the permutations
    candidate_sequence = ""
    for i in range(0,len(recoded_sequence),3):
        aa = codon_to_aa_dict[recoded_sequence[i:i+3]]
        pos = aa_permuted_positions[aa].index(i)
        if (aa == "C"):
            print(i, " ", pos)
        candidate_sequence += recoded_sequence[pos:pos+3]
    candidate_cps = get_cps_indep(candidate_sequence)
    if(candidate_cps > best_so_far_indep_cps):
        best_so_far_sequence = candidate_sequence
        best_so_far_indep_cps = candidate_cps

back_to_recoded = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (best_so_far_sequence[i:i+3] != recoded_sequence[i:i+3]):
        diff_total += 1
    back_to_recoded += codon_to_aa_dict[best_so_far_sequence[i:i+3]]
if (original_AA == back_to_recoded):
    print("AA sequence remains the same, good")
else:
    print(original_AA)
    print(back_to_recoded)

print("coleman codon pair bias of the candidate sequence: " + str(get_cpb(candidate_sequence)))
print("frame2&3 indep codon pair score of the candidate sequence: " + str(get_cps_indep(candidate_sequence)))

total number of codon diferences between original and recoded sequences 576
AA sequence remains the same, good
coleman codon pair bias of the original sequence: -0.03513233601841192
coleman codon pair bias of the recoded sequence: -0.036011507479861975
frame2&3 indep codon pair score of the original sequence: -31.61352552200005
frame2&3 indep codon pair score of the recoded sequence: -488.30513615799987
C
[195, 357, 510, 567, 699, 789, 945, 948, 1059, 1242, 1353, 1482, 1641, 1962, 2514]
[2514, 357, 1242, 1353, 195, 945, 1482, 1641, 510, 948, 1962, 1059, 567, 789, 699]
195   4
357   1
510   8
567   12
699   14
789   13
945   5
948   9
1059   11
1242   2
1353   3
1482   6
1641   7
1962   10
2514   0
GAHENSNRAYGSSTINYTTINYYRDSASNAASKQDFSQDPSKFTEPIKDVLIKTAPMLNSPNIEACGYSDRVLQLTLGNSTITTQEAANSVVAYGRWPEYLRDSEANPVDQPTEPDVAACRFYTLDTVSWTKESRGWWWKLPDALRDMGLFGQNMYYHYLGRSGYTVHVQCNASKFHQGALGVFAVPEMCLAGDSNTTTMHTSYQNANPGEKGGTFTGTFTPDNNQTSPARRFCPVDYLLGNGTLLGNAFVFPHQIINLRTNNCATLVLPYVNSLSIDSMVKHNNWGIAILPL